In [1]:
dataset_id = "reporting"
!bq --location=US mk --dataset {dataset_id}

Dataset 'airy-lodge-252816:reporting' successfully created.


-- Query 1 --
-- The expected results of this query would be the player ids of all players that participated in every game played in the NFL. Furthermore this query will yield the home and away team, the scores of the home and away team, the stadium name, and the year the game was played.

In [5]:
%%bigquery
create view reporting.v_players_who_played_all_games as
select a.player_id, b.stadium, b.team_home, b.team_away, b.score_home, b.score_away, b.schedule_season
from `airy-lodge-252816.nfl_stats_modeled.Quarterback_Game_Logs_Beam_DF` a join `airy-lodge-252816.nfl_weather_modeled.spreadspoke_scores` b on a.game_date = b.schedule_date
where a.year = b.schedule_season

""


In [7]:
%%bigquery
select count(*) from reporting.v_players_who_played_all_games

,f0_
0,387395


-- Query 2 --
-- This query's expected results would be to get the average passing yards of QBs when playing in a covered stadium

In [2]:
%%bigquery
create view reporting.v_average_passing_yards_in_dome as
select q.player_id, avg(passing_yards) as avg_passing_yards
from `airy-lodge-252816.nfl_stats_modeled.Quarterback_Game_Logs_Beam_DF` as q join `airy-lodge-252816.nfl_weather_modeled.spreadspoke_scores` as s on q.game_date = s.schedule_date
where s.weather_detail = "DOME"
group by q.player_id
order by avg_passing_yards desc

""


--Query 3--
-- This query's expected results would be to get the average rushing yards of each player when the temperature was less than 20 degrees. 

In [14]:
%%bigquery
create view reporting.v_average_rushing_yards_in_cold_weather as
select a.player_id, round(avg(a.rushing_yards), 2) as avg_rushing_yards
from `airy-lodge-252816.nfl_stats_modeled.Quarterback_Game_Logs_Beam_DF` a join `airy-lodge-252816.nfl_weather_modeled.spreadspoke_scores` b on a.game_date = b.schedule_date 
where a.year = b.schedule_season
and  b.weather_temperature < 20.0
group by a.player_id

""


In [15]:
%%bigquery
select count(*) from reporting.v_average_rushing_yards_in_cold_weather

,f0_
0,370
